### 1)-Importing Packages

In [1]:
import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Row, SQLContext}
//import org.apache.spark.ml.clustering.{KMeans, KMeansModel, GaussianMixture, GaussianMixtureModel, BisectingKMeans, BisectingKMeansModel}
import org.apache.spark.ml.clustering._
import org.apache.spark.ml.evaluation.ClusteringEvaluator
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator,BinaryClassificationEvaluator}
import org.apache.spark.ml.feature.{StandardScaler, VectorAssembler, StringIndexer, MinMaxScaler}
import org.apache.spark.ml.{Pipeline}
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import java.io.{File, PrintWriter}
import java.text.SimpleDateFormat
import java.util.Calendar

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
78,application_1576053140655_0079,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Row, SQLContext}
import org.apache.spark.ml.clustering._
import org.apache.spark.ml.evaluation.ClusteringEvaluator
import org.apache.spark.ml.evaluation.{MulticlassClassificationEvaluator, BinaryClassificationEvaluator}
import org.apache.spark.ml.feature.{StandardScaler, VectorAssembler, StringIndexer, MinMaxScaler}
import org.apache.spark.ml.Pipeline
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import java.io.{File, PrintWriter}
import java.text.SimpleDateFormat
import java.util.Calendar


### 2)-Load Dataset

In [2]:
val raw = spark.read.format("csv").option("header", "true").option("mode", "DROPMALFORMED").csv("datasets/creditcard.csv")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

raw: org.apache.spark.sql.DataFrame = [Time: string, V1: string ... 29 more fields]


In [3]:
// cast all the column to Double type.
val df = raw.select(((1 to 28).map(i => "V" + i) ++ Array("Time", "Amount", "Class")).map(s => col(s).cast("Double")): _*)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 29 more fields]


### 3)-Data Preparation for Training

In [4]:
val labelConverter = new StringIndexer().setInputCol("Class").setOutputCol("label")
val assembler = new VectorAssembler().setInputCols(Array("V3", "V4", "V9", "V10", "V11", "V12", "V14", "V16", "V17", "V18","V19")).setOutputCol("assembled")
val scaler = new MinMaxScaler().setInputCol("assembled").setOutputCol("features")
//val scaler = new StandardScaler().setInputCol("assembled").setOutputCol("features")
val pipeline = new Pipeline().setStages(Array(assembler, scaler, labelConverter))
val pipelineModel = pipeline.fit(df)
val data = pipelineModel.transform(df)
println("Generate feature from raw data:")
data.select("features", "label").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

labelConverter: org.apache.spark.ml.feature.StringIndexer = strIdx_805a99f48742
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_9d9e732dd7fe
scaler: org.apache.spark.ml.feature.MinMaxScaler = minMaxScal_992322055318
pipeline: org.apache.spark.ml.Pipeline = pipeline_71b17bd65f89
pipelineModel: org.apache.spark.ml.PipelineModel = pipeline_71b17bd65f89
data: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 32 more fields]
Generate feature from raw data:
+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.88136490328633...|  0.0|
|[0.84029849039390...|  0.0|
|[0.86814081926190...|  0.0|
|[0.86848364774806...|  0.0|
|[0.86425070140685...|  0.0|
|[0.85718742663956...|  0.0|
|[0.83819983804773...|  0.0|
|[0.85603110871019...|  0.0|
|[0.83545216735689...|  0.0|
|[0.85551101187934...|  0.0|
|[0.85324951634142...|  0.0|
|[0.82226325870284...|  0.0|
|[0.84406658980857...|  0.0|
|[0.85177230541965...|  0.0|
|[0.86586281889129

In [5]:
val data0 = data.filter($"Class" === "0").cache()
val data1 = data.filter($"Class" === "1").cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

data0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
data1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]


In [6]:
val splitTime0 = data0.stat.approxQuantile("Time", Array(0.7), 0.001).head
val splitTime1 = data1.stat.approxQuantile("Time", Array(0.6), 0.001).head
val trainingData0 = data0.filter(s"Time<$splitTime0").cache()
val validationData0 = data0.filter(s"Time>=$splitTime0").cache()
val trainingData1 = data1.filter(s"Time<$splitTime1").cache()
val validationData1 = data1.filter(s"Time>=$splitTime1").cache()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

splitTime0: Double = 132934.0
splitTime1: Double = 93824.0
trainingData0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData0: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
trainingData1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]


In [7]:
val trainingData = trainingData0.unionAll(trainingData1)
val validationData = validationData0.unionAll(validationData1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

trainingData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]
validationData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [V1: double, V2: double ... 32 more fields]


In [8]:
println(" Training set statistics: 1 represents fraud and 0 represents normal")
trainingData.groupBy("Class").count().show()
println(" validation set statistics: 1 represents fraud and 0 represents normal")
validationData.groupBy("Class").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 Training set statistics: 1 represents fraud and 0 represents normal
+-----+------+
|Class| count|
+-----+------+
|  0.0|198993|
|  1.0|   295|
+-----+------+

 validation set statistics: 1 represents fraud and 0 represents normal
+-----+-----+
|Class|count|
+-----+-----+
|  0.0|85322|
|  1.0|  197|
+-----+-----+



### 4)-Training the models

In [9]:
// Create a Kmeans Model with K=2
// train the model
val kmeans = new KMeans().setK(2).setFeaturesCol("features").setPredictionCol("clusters").setSeed(1L).setMaxIter(100)
val bisectingkmeans = new BisectingKMeans().setK(2).setFeaturesCol("features").setPredictionCol("clusters").setSeed(1L).setMaxIter(100)
val gaussianMixture = new GaussianMixture().setK(2).setFeaturesCol("features").setPredictionCol("clusters").setSeed(1L).setMaxIter(100)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

kmeans: org.apache.spark.ml.clustering.KMeans = kmeans_ff1415de2ad0
bisectingkmeans: org.apache.spark.ml.clustering.BisectingKMeans = bisecting-kmeans_19bffe496895
gaussianMixture: org.apache.spark.ml.clustering.GaussianMixture = GaussianMixture_c33ee33bcb7f


In [10]:
// Fit that model to the training_data
val t = System.nanoTime
val modelKmeans = kmeans.fit(trainingData)
val durationkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationkmeans secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 358400929306774
modelKmeans: org.apache.spark.ml.clustering.KMeansModel = kmeans_ff1415de2ad0
durationkmeans: Double = 9.859007144

initial model training finished.
Training process takes 9.859007144 secs


In [11]:
// Fit that model to the training_data
val t = System.nanoTime
val modelBisectingkmeans = bisectingkmeans.fit(trainingData)
val durationbisectingkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationbisectingkmeans secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 358412208709298
modelBisectingkmeans: org.apache.spark.ml.clustering.BisectingKMeansModel = bisecting-kmeans_19bffe496895
durationbisectingkmeans: Double = 37.809269994

initial model training finished.
Training process takes 37.809269994 secs


In [12]:
// Fit that model to the training_data
val t = System.nanoTime
val modelGMM = gaussianMixture.fit(trainingData)
val durationGMM = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationGMM secs")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 358451818381165
modelGMM: org.apache.spark.ml.clustering.GaussianMixtureModel = GaussianMixture_c33ee33bcb7f
durationGMM: Double = 10.75590278

initial model training finished.
Training process takes 10.75590278 secs


### 5)- Prediction

In [13]:
 // Make predictions Kmeans
val t = System.nanoTime
var predictionsk = modelKmeans.transform(validationData)
val durationkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationkmeans secs")
predictionsk = predictionsk.select(col("label"),col("clusters").cast("Double"),col("features"))
//val df = raw.select(((1 to 28).map(i => "V" + i) ++ Array("Time", "Amount", "Class")).map(s => col(s).cast("Double")): _*)
predictionsk.groupBy("clusters").count().show()
//predictionsk.groupBy("label").count().show()
predictionsk.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 358463390988614
predictionsk: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 33 more fields]
durationkmeans: Double = 0.246136773

initial model training finished.
Training process takes 0.246136773 secs
predictionsk: org.apache.spark.sql.DataFrame = [label: double, clusters: double ... 1 more field]
+--------+-----+
|clusters|count|
+--------+-----+
|     0.0|48514|
|     1.0|37005|
+--------+-----+

+-----+--------+--------------------+
|label|clusters|            features|
+-----+--------+--------------------+
|  0.0|     1.0|[0.83928824494586...|
|  0.0|     1.0|[0.84019343593776...|
|  0.0|     1.0|[0.78506093453763...|
|  0.0|     1.0|[0.86327731276444...|
|  0.0|     1.0|[0.81905350742297...|
|  0.0|     0.0|[0.80576713299952...|
|  0.0|     0.0|[0.75665670211160...|
|  0.0|     1.0|[0.81390257419240...|
|  0.0|     1.0|[0.82786727703355...|
|  0.0|     0.0|[0.84443102534225...|
+-----+--------+--------------------+
only showing top 10 rows



In [14]:
predictionsk.select("clusters", "label").groupBy("clusters", "label").count().orderBy("label", "clusters").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+-----+
|clusters|label|count|
+--------+-----+-----+
|     0.0|  0.0|48476|
|     1.0|  0.0|36846|
|     0.0|  1.0|   38|
|     1.0|  1.0|  159|
+--------+-----+-----+



In [15]:
 // Make predictions Bisecting Kmeans 
val t = System.nanoTime
var predictionsbk = modelBisectingkmeans.transform(validationData)
val durationkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationkmeans secs")
predictionsbk = predictionsbk.select(col("label"),col("clusters").cast("Double"),col("features"))
predictionsbk.groupBy("clusters").count().show()
predictionsbk.groupBy("label").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 358467894947030
predictionsbk: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 33 more fields]
durationkmeans: Double = 0.235729777

initial model training finished.
Training process takes 0.235729777 secs
predictionsbk: org.apache.spark.sql.DataFrame = [label: double, clusters: double ... 1 more field]
+--------+-----+
|clusters|count|
+--------+-----+
|     0.0|48511|
|     1.0|37008|
+--------+-----+

+-----+-----+
|label|count|
+-----+-----+
|  0.0|85322|
|  1.0|  197|
+-----+-----+



In [16]:
predictionsbk.select("clusters", "label").groupBy("clusters", "label").count().orderBy("label", "clusters").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+-----+
|clusters|label|count|
+--------+-----+-----+
|     0.0|  0.0|48473|
|     1.0|  0.0|36849|
|     0.0|  1.0|   38|
|     1.0|  1.0|  159|
+--------+-----+-----+



In [17]:
// Make predictions Bisecting Kmeans 
val t = System.nanoTime
var predictionsgmm = modelGMM.transform(validationData)
val durationkmeans = (System.nanoTime - t) / 1e9d
println("\ninitial model training finished.")
println(s"Training process takes $durationkmeans secs")
predictionsgmm = predictionsgmm.select(col("label"),col("clusters").cast("Double"),col("features"))
predictionsgmm.groupBy("clusters").count().show()
predictionsgmm.groupBy("label").count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

t: Long = 358472399412990
predictionsgmm: org.apache.spark.sql.DataFrame = [V1: double, V2: double ... 34 more fields]
durationkmeans: Double = 0.237759191

initial model training finished.
Training process takes 0.237759191 secs
predictionsgmm: org.apache.spark.sql.DataFrame = [label: double, clusters: double ... 1 more field]
+--------+-----+
|clusters|count|
+--------+-----+
|     0.0|62153|
|     1.0|23366|
+--------+-----+

+-----+-----+
|label|count|
+-----+-----+
|  0.0|85322|
|  1.0|  197|
+-----+-----+



In [18]:
predictionsgmm.select("clusters", "label").groupBy("clusters", "label").count().orderBy("label", "clusters").show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-----+-----+
|clusters|label|count|
+--------+-----+-----+
|     0.0|  0.0|62105|
|     1.0|  0.0|23217|
|     0.0|  1.0|   48|
|     1.0|  1.0|  149|
+--------+-----+-----+



### 6)-Evaluation

 ##### Calculation of Silhouette Score

In [19]:
// Evaluate clustering by computing Silhouette score
val evaluatorsi = new ClusteringEvaluator().setPredictionCol("clusters").setMetricName("silhouette")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluatorsi: org.apache.spark.ml.evaluation.ClusteringEvaluator = cluEval_557d93c04e88


In [20]:
val ScoreKmeans = evaluatorsi.evaluate(predictionsk)
println(s"Silhouette of Kmeans predictions with squared euclidean distance = $ScoreKmeans")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ScoreKmeans: Double = 0.24075762081495494
Silhouette of Kmeans predictions with squared euclidean distance = 0.24075762081495494


In [21]:
val ScoreBisectingKmeans  = evaluatorsi.evaluate(predictionsbk)
println(s"Silhouette of Bisecting Kmeans predictions with squared euclidean distance = $ScoreBisectingKmeans")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ScoreBisectingKmeans: Double = 0.24104279162878414
Silhouette of Bisecting Kmeans predictions with squared euclidean distance = 0.24104279162878414


In [22]:
val ScoreGMM  = evaluatorsi.evaluate(predictionsgmm)
println(s"Silhouette of Gaussian Mixture predictions with squared euclidean distance = $ScoreGMM")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ScoreGMM: Double = 0.09193997360784865
Silhouette of Gaussian Mixture predictions with squared euclidean distance = 0.09193997360784865


In [23]:
// Evaluate clustering.
val costKmeans = modelKmeans.computeCost(trainingData)
println(s"Kmeans: Within Set Sum of Squared Errors = $costKmeans")
val costBisectingKmeans = modelBisectingkmeans.computeCost(trainingData)
println(s"Bisecting Kmeans: Within Set Sum of Squared Errors = $costBisectingKmeans")
val costBisectingKmeans = modelBisectingkmeans.computeCost(trainingData)
println(s"Bisecting Kmeans: Within Set Sum of Squared Errors = $costBisectingKmeans")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

costKmeans: Double = 3781.2986393374727
Kmeans: Within Set Sum of Squared Errors = 3781.2986393374727
costBisectingKmeans: Double = 3781.2930628987174
Bisecting Kmeans: Within Set Sum of Squared Errors = 3781.2930628987174
costBisectingKmeans: Double = 3781.2930628987183
Bisecting Kmeans: Within Set Sum of Squared Errors = 3781.2930628987183


In [24]:
val evaluatork1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("accuracy")
val evaluatork2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedPrecision")
val evaluatork3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedRecall")
val evaluatork4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("f1")
val areaUnderROCk = new BinaryClassificationEvaluator().setRawPredictionCol("clusters").setLabelCol("label").setMetricName("areaUnderROC")
val accuracyk = evaluatork1.evaluate(predictionsk)
println("Area Under ROC Curve = " + areaUnderROCk.evaluate(predictionsk))
println("Accuracy = " + evaluatork1.evaluate(predictionsk))
println("Precision = " + evaluatork2.evaluate(predictionsk))
println("Recall = " + evaluatork3.evaluate(predictionsk))
println("F1 = " + evaluatork4.evaluate(predictionsk))
println("Test Error = " + (1.0 - accuracyk))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluatork1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_b7a2be46a0d5
evaluatork2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_7fc82fcfa9d3
evaluatork3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_ba2e5f8d8b9e
evaluatork4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_9b896a9e586c
areaUnderROCk: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_4b6b06777912
accuracyk: Double = 0.5687040306832399
Area Under ROC Curve = 0.68763009094125
Accuracy = 0.5687040306832399
Precision = 0.9969248414712742
Recall = 0.56870403068324
F1 = 0.7227599337648909
Test Error = 0.43129596931676006


In [25]:
val evaluatorbk1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("accuracy")
val evaluatorbk2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedPrecision")
val evaluatorbk3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedRecall")
val evaluatorbk4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("f1")
val areaUnderROCbk = new BinaryClassificationEvaluator().setRawPredictionCol("clusters").setLabelCol("label").setMetricName("areaUnderROC")
val accuracybk = evaluatorbk1.evaluate(predictionsbk)
println("Area Under ROC Curve = " + areaUnderROCbk.evaluate(predictionsbk))
println("Accuracy = " + evaluatorbk1.evaluate(predictionsbk))
println("Precision = " + evaluatorbk2.evaluate(predictionsbk))
println("Recall = " + evaluatorbk3.evaluate(predictionsbk))
println("F1 = " + evaluatorbk4.evaluate(predictionsbk))
println("Test Error = " + (1.0 - accuracybk))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluatorbk1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_685793439152
evaluatorbk2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_97c841d9dec5
evaluatorbk3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_5e5afc19d259
evaluatorbk4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_12cc15ec6ff6
areaUnderROCbk: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_f6d28b6cbdab
accuracybk: Double = 0.5686689507594803
Area Under ROC Curve = 0.6876125104813453
Accuracy = 0.5686689507594803
Precision = 0.9969247923412398
Recall = 0.5686689507594803
F1 = 0.7227314044016117
Test Error = 0.4313310492405197


In [26]:
val evaluatorgmm1 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("accuracy")
val evaluatorgmm2 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedPrecision")
val evaluatorgmm3 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("weightedRecall")
val evaluatorgmm4 = new MulticlassClassificationEvaluator().setLabelCol("label").setPredictionCol("clusters").setMetricName("f1")
val areaUnderROCgmm = new BinaryClassificationEvaluator().setRawPredictionCol("clusters").setLabelCol("label").setMetricName("areaUnderROC")
val accuracygmm = evaluatorgmm1.evaluate(predictionsgmm)
println("Area Under ROC Curve = " + areaUnderROCgmm.evaluate(predictionsgmm))
println("Accuracy = " + evaluatorgmm1.evaluate(predictionsgmm))
println("Precision = " + evaluatorgmm2.evaluate(predictionsgmm))
println("Recall = " + evaluatorgmm3.evaluate(predictionsgmm))
println("F1 = " + evaluatorgmm4.evaluate(predictionsgmm))
println("Test Error = " + (1.0 - accuracygmm))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

evaluatorgmm1: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_6c0130fc2db5
evaluatorgmm2: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_0861cda3b105
evaluatorgmm3: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_9f7d709fbeec
evaluatorgmm4: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_821bec04a822
areaUnderROCgmm: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_a0701b80d8f3
accuracygmm: Double = 0.7279551912440511
Area Under ROC Curve = 0.7421174096289994
Accuracy = 0.7279551912440511
Precision = 0.9969405990752082
Recall = 0.7279551912440511
F1 = 0.8403334026212206
Test Error = 0.27204480875594894
